In [1]:
import os
import shutil
from pathlib import Path
from datetime import datetime

from dotenv import load_dotenv
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
import requests
from supabase import create_client, Client
import instaloader
from models import VideoAssetForm, Restaurant

import mux_python
from mux_python.rest import ApiException


# Load environment variables
load_dotenv()

# Constants
SUPABASE_URL: str = os.environ.get('NICO_SUPABASE_URL')
SUPABASE_KEY: str = os.environ.get('NICO_API_KEY')
SUPABASE_BUCKET: str = 'bucket'
RESTAURANT_CONCEPT_TABLE: str = "restaurant_concept"
VIDEO_METADATA_TABLE: str = "video_meta_data"
DOWNLOADS_DIR: Path = Path('downloads')

configuration = mux_python.Configuration()
configuration.username = os.getenv('MUX_TOKEN_ID')
configuration.password = os.getenv('MUX_TOKEN_SECRET')

def handle_instagram_download(post_url: str) -> Path | None:
    """Download video from Instagram post URL.
    
    Args:
        post_url (str): The URL of the Instagram post.

    Returns:
        Union[Path, None]: The path to the downloaded video or None if not found.
    """
    loader = instaloader.Instaloader(download_pictures=False, download_videos=True, download_video_thumbnails=False, download_geotags=False, download_comments=False)
    shortcode = post_url.split("/")[-2]
    post = instaloader.Post.from_shortcode(loader.context, shortcode)

    if not post.is_video:
        return None

    target_directory = Path(f"downloads/{post.shortcode}")
    target_directory.mkdir(parents=True, exist_ok=True)
    loader.download_post(post, target=target_directory)

    for file in target_directory.iterdir():
        if file.is_file() and file.suffix == '.mp4':
            return file

    return None

def delete_file_and_siblings(file_path: Path):
    # Delete the file
    file_path.unlink()
    
    # Get the parent directory
    parent_dir = file_path.parent
    
    # Delete all sibling files
    for sibling_file in parent_dir.iterdir():
        if sibling_file.is_file():
            sibling_file.unlink()
            
    # Delete the parent directory
    shutil.rmtree(parent_dir)

# Initialize Supabase client
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

ImportError: cannot import name 'VideoAssetForm' from 'models' (c:\Users\nickb\OneDrive\Documents\B2C_Tools\downloaderAPI\models.py)

In [ ]:
# Create a Mux API client for direct uploads
direct_uploads_api = mux_python.DirectUploadsApi(mux_python.ApiClient(configuration))

# Create a new Direct Upload
create_asset_request = mux_python.CreateAssetRequest(playback_policy=[mux_python.PlaybackPolicy.PUBLIC])
create_upload_request = mux_python.CreateUploadRequest(new_asset_settings=create_asset_request, cors_origin='*')
create_upload_response = direct_uploads_api.create_direct_upload(create_upload_request)


In [ ]:
restaurant_concept_data, count = supabase.table("restaurant_concept")\
        .select("*")\
        .eq("concept_name", "restaurant ee")\
        .execute()

        

if len(restaurant_concept_data[1]) < 1:
    # # Insert a new concept if none was found
    restaurant_concept_data, count = supabase.table("restaurant_concept").insert({"concept_name": "restaurant_concept"}).execute()

restaurant_concept_id = restaurant_concept_data[1][0]['restaurant_concept_id']



downloaded_video = handle_instagram_download("https://www.instagram.com/reel/C1VBM5RxoAP/")

In [18]:

# print("video: ", downloaded_video)x``
import time


if downloaded_video is not None:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    new_video_filename = f"{downloaded_video.stem}_{timestamp}{downloaded_video.suffix}"

    # Upload video to bucket
    # supabase.storage.from_("videos").upload(new_video_filename, downloaded_video)
    
    
    # Get the authenticated URL
    upload_url = create_upload_response.data.url

    # Upload the video file
    with open(downloaded_video, 'rb') as file:
        requests.put(upload_url, data=file)

    # Wait for the asset to become ready
    assets_api = mux_python.AssetsApi(mux_python.ApiClient(configuration))
    upload_response = direct_uploads_api.get_direct_upload(create_upload_response.data.id)

    assert upload_response != None
    assert upload_response.data.id != None
    assert upload_response.data.id == create_upload_response.data.id

    mux_asset_id = upload_response.data.asset_id

    # Get the asset details
    asset = assets_api.get_asset(mux_asset_id)

    # Now you can access the playback_ids
    mux_playback_id = asset.data.playback_ids[0].id
    
    # Get the Mux Playback ID
    # mux_playback_id = upload_response.data.playback_ids[0].id

    # public_url = supabase.storage.from_("videos").get_public_url(new_video_filename)
    # delete_file_and_siblings(downloaded_video)
    

ApiValueError: Missing the required parameter `asset_id` when calling `get_asset`

In [ ]:
print(mux_playback_id)
print(mux_asset_id)

In [ ]:
print(upload_response.data)